# Pushing Block Simulation

Importing packages

In [95]:
try: 
    from jax import config
    config.update("jax_enable_x64", True)
    import time
    import jax
    import jax.numpy as jnp
    from jax import jacfwd
    from meshcat import Visualizer
    import meshcat.geometry as mc_geom
    import meshcat.transformations as mc_trans



    print('Imported packages.')
except Exception as e:
    print('Importing packages failed:')
    print(e)



Imported packages.


Integrator and constants.

In [96]:
R = 0.1 # meter
m_ee = 0.2 # kg
m_block = 1.0 # kg
a_block = 1.0 # meter
q_init = [2.0, 0.0] # end effector, block : center x pos in meters
qdot_init = [0., 0.] # end effector, block : x vel in meters/dt
M = jnp.diag(jnp.array([m_ee, m_block]))
b = jnp.array([-1., 0.])
k = 1e4
c = 10

# Runge-Kutta 4
def rk4_step(f, x, dt):
    """
        Input:
            f(x) - function to be integrated
            x - current state
            dt - time step 
        Output: 
            x[t+dt] 
    """
    k1 = f(x)
    k2 = f(x + dt*k1/2)
    k3 = f(x + dt*k2/2)
    k4 = f(x + dt*k3)
    xdot = x + 1/6*(k1+2*k2+2*k3+k4)*dt
    return xdot 

Contact jacobian

In [ ]:
def phi(q):
    return jnp.array([abs(q[0] - q[1]) - (R + a_block / 2)])

J_c = jacfwd(phi)

def contact_model(phi, v):
    lam = k * jnp.clip(-phi, 0.0, None) + c * jnp.clip(-v, 0.0, None)
    return lam  

The following function computes the dynamics of the system each timestep.

In [98]:
J_c = jax.jit(J_c)
contact_model = jax.jit(contact_model)

def dyn_step(x):
    """
        Input: 
            state = x = [q, qdot] 
        Output: 
            xdot = [qdot, qddot]
    """
    q, qdot = jnp.split(x, 2)
    F = 0.0
    if phi(q) <= 0:
        F = J_c(q).T @ contact_model(phi(q), J_c(q) @ qdot)
    qddot = jnp.linalg.solve(M, (F + b))
    return jnp.hstack(jnp.array([qdot, qddot]))

Initialize the visualizer.

In [99]:
class Viewer1D(Visualizer):
    def __init__(self, R, a, q_init) -> None:
        Visualizer.__init__(self)
        self._block = self["block"]
        self._end_effector = self["end_effector"]
        self._end_effector.set_object(
            mc_geom.Sphere(radius=R),
            mc_geom.MeshLambertMaterial(
                    color=0x0000ff,
                    # opacity=0.5,
                    reflectivity=0.8,
                    )
        )
        self._block.set_object(
            mc_geom.Box([a, a, a]),
            mc_geom.MeshLambertMaterial(
                    color=0x00ff00,
                    # opacity=0.5,
                    reflectivity=0.8,
            )
        )
        self.render(q_init)
    def render(self, q):
            x_ee, x_block = q
            _T_ee = mc_trans.compose_matrix(
                            translate=[0.,x_ee,0], 
                            angles=[0.,0.,0.]
                )
            _T_block = mc_trans.compose_matrix(
                            translate=[0.,x_block,0], 
                            angles=[0.,0.,0.]
                )
            self._end_effector.set_transform(_T_ee)
            self._block.set_transform(_T_block)

viewer = Viewer1D(R, a_block, q_init)
viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


## Animation

Using RK4 for integration to propagate.

In [100]:
tf = 4
dt = 1e-2
N = int(tf/dt)
x0 = jnp.array(q_init + qdot_init)
for k in range(N):
    x0 = rk4_step(dyn_step, x0, dt)
    q,qdot = jnp.split(x0, 2)
    viewer.render(q)
    time.sleep(dt)